In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

input\items.csv
input\item_categories.csv
input\sales_train.csv
input\sample_submission.csv
input\shops.csv
input\test.csv


# Data analysis

## What needs to be done?
The task is to forecast the total amount of products sold in every shop for the test set.

! Note that the list of shops and products slightly changes every month.

### Pipeline 
* supervised learning task (we have the labeled training examples) 
* multiple regression problem (value prediction) - the system will use multiple features to make a prediction
* univariate regression problem - we're only trying to predict a single value in every shop
* plain batch learning - we don't have a continuous flow of data coming to the system - the data doesn't need to be adjusted rapidly, and the data is small enoug to fit in memory `is it?`

In [15]:
import csv
import os

ifile = os.path.abspath(os.path.join('input', 'sales_train.csv'))
with open(ifile) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        print(row)

 '6738', '699.0', '3.0']
['21.03.2013', '2', '14', '6738', '699.0', '2.0']
['20.03.2013', '2', '14', '6738', '699.0', '2.0']
['19.03.2013', '2', '14', '6738', '699.0', '1.0']
['18.03.2013', '2', '14', '6738', '699.0', '2.0']
['17.03.2013', '2', '14', '6738', '699.0', '1.0']
['16.03.2013', '2', '14', '6738', '699.0', '7.0']
['15.03.2013', '2', '14', '6738', '699.0', '1.0']
['14.03.2013', '2', '14', '6738', '699.0', '2.0']
['13.03.2013', '2', '14', '6738', '699.0', '5.0']
['23.03.2013', '2', '14', '6738', '699.0', '4.0']
['12.03.2013', '2', '14', '6738', '699.0', '18.0']
['31.03.2013', '2', '14', '6724', '419.0', '1.0']
['21.03.2013', '2', '14', '6721', '1499.0', '1.0']
['30.03.2013', '2', '14', '6717', '299.0', '1.0']
['25.03.2013', '2', '14', '6717', '299.0', '2.0']
['07.03.2013', '2', '14', '6717', '299.0', '1.0']
['02.03.2013', '2', '14', '6715', '419.0', '1.0']
['24.03.2013', '2', '14', '6708', '418.7', '1.0']
['19.03.2013', '2', '14', '6708', '418.7', '1.0']
['14.03.2013', '2', '14

KeyboardInterrupt: 

## Input files

* shops.csv- supplemental information about the shops -- `61 entries`
    * shop_name (e.g., "СПб ТК ""Сенная""")
    * shop_id (e.g., 43)

* item_categories.csv  - supplemental information about the items categories -- `85 entries`
    * item_category_name (e.g., Кино - DVD)
    * item_category_id (e.g., 40)

* items.csv - supplemental information about the items/products -- `22.171 entries`
    * item_name (e.g., 1812: 4 СЕРИИ (регион))
    * item_id (e.g., 97)
    * item_category_id (e.g., 40)

* sales_train.csv - the training set. Daily historical data from January 2013 to October 2015 -- `2.935.850 entries | 587.170 entries should be allotted to the training set`
    * <strike>date (e.g., 23.02.2013)</strike> *I don't see the reason of using this in ML training because we already have date_block_num as an attribute*
    * date_block_num (e.g., 1)
    * shop_id (e.g., 43) - `shop_id and item_id shall be concatenated to ID`
    * item_id (e.g., 97) - `shop_id and item_id shall be concatenated to ID`
    * item_price (e.g., 149.0)
    * item_cnt_day (e.g., 1.0)

* sample_submission.csv - a sample submission file in the correct format -- `214.201 entries`
    * ID (e.g., 0)
    * item_cnt_month (e.g., 0.5)

* test.csv - the test set. You need to forecast the sales for these shops and products for November 2015 -- `214.201 entries`
    * ID (e.g., 0)
    * shop_id (e.g., 43)
    * item_id (e.g., 97)

## Data fields
* ID - an Id that represents a (Shop, Item) tuple within the test set
* shop_id - unique identifier of a shop
* item_id - unique identifier of a product
* item_category_id - unique identifier of item category
* item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
* item_price - current price of an item
* date - date in format dd/mm/yyyy
* date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* item_name - name of item
* shop_name - name of shop
* item_category_name - name of item category

# Model

tbd